# XGBoost

Testing out XGBoost and it did not work lol

In [31]:
print("Importing...")
from pprint import pprint
import json
import sys
import os
from shutil import rmtree
from copy import deepcopy
from os.path import join
from importlib import reload

from os.path import isdir
from os.path import join
from os import mkdir
from shutil import rmtree

import xgboost
import numpy as np
import pandas as pd
from scipy import stats
from matplotlib import pyplot as plt
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score
from tensorflow.keras.utils import to_categorical

sys.path.insert(0, os.path.join(os.getcwd(), "../scs"))
import scs_config as scsc
import data_loading as dl
import data_degrading as dd
import data_preparation as dp
import data_augmentation as da
import data_plotting as dplt
import learn
import lr_schedules
import hp_sets
import prepare_dataset

Importing...


In [3]:
R = 100
data_dir_original = "/home/2649/repos/SCS/data/"
dir_models = "/lustre/lrspec/users/2649/models/transformer_testing"

# Construct the directories if they don't exist or delete them and recreate
# them if they do and `restart_fit` is `True`.
dir_model = join(dir_models, f"{R}_xgboost")
dir_backup = join(dir_model, "backup")
dir_model_data = join(dir_model, "data")
if isdir(dir_model):
    rmtree(dir_model)
mkdir(dir_model)
mkdir(dir_backup)
mkdir(dir_model_data)

file_trn = join(dir_model_data, f"sn_data_trn.RPA.parquet")
file_tst = join(dir_model_data, f"sn_data_tst.RP.parquet")

In [4]:
hp = deepcopy(scsc.default_hyper_parameters)

hp["train_frac"] = 0.80
hp["noise_scale"] = 0.15848931924611134
hp["spike_scale"] = 1.045639552591273
hp["max_spikes"] = 3

In [5]:
# Prepare the dataset from the original dataset dataframe `sn_data_file`.
sn_data_file = join(data_dir_original, "sn_data.parquet")
prepare_dataset.prepare_dataset(
    R,
    sn_data_file,
    dir_model_data,
    dir_model_data,
    dir_model_data,
    dir_model_data,
    hp["phase_range"],
    hp["ptp_range"],
    hp["wvl_range"],
    hp["train_frac"],
    hp["noise_scale"],
    hp["spike_scale"],
    hp["max_spikes"],
    random_state=hp["random_state"],
)

# Load the dataset.
df_trn = dl.load_sn_data(file_trn)
df_tst = dl.load_sn_data(file_tst)
dataset, num_wvl, num_classes = learn.prepare_datasets_for_training(
    df_trn, df_tst
)
Xtrn, Ytrn, Xtst, Ytst = dataset

Loaded: /home/2649/repos/SCS/data/sn_data.parquet
Artificially degrading the following dataset to a spectral resolution of 100:
    /home/2649/repos/SCS/data/sn_data.parquet
Saved: /lustre/lrspec/users/2649/models/transformer_testing/100_xgboost/data/sn_data.C.parquet
Saved: /lustre/lrspec/users/2649/models/transformer_testing/100_xgboost/data/sn_data.R.parquet
Degrading completed.

Preprocesing the following dataset files:
    /lustre/lrspec/users/2649/models/transformer_testing/100_xgboost/data/sn_data.C.parquet
    /lustre/lrspec/users/2649/models/transformer_testing/100_xgboost/data/sn_data.R.parquet
Saved: /lustre/lrspec/users/2649/models/transformer_testing/100_xgboost/data/sn_data.CP.parquet
Saved: /lustre/lrspec/users/2649/models/transformer_testing/100_xgboost/data/sn_data.RP.parquet
Preprocessing complete.

Performing special train-test split on the following dataset files:
    /lustre/lrspec/users/2649/models/transformer_testing/100_xgboost/data/sn_data.CP.parquet
    /lustr

In [32]:
xgb = xgboost.XGBClassifier(
    n_estimators=100,
    max_depth=5,
    # learning_rate=1e-3,
    verbosity=1,
    objective="binary:logistic"
)
xgb.fit(Xtrn[..., 0], np.argmax(Ytrn, axis=1))

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [33]:
print(xgb.score(Xtrn[..., 0], np.argmax(Ytrn, axis=1)))
print(xgb.score(Xtst[..., 0], np.argmax(Ytst, axis=1)))

Ptrn = xgb.predict(Xtrn[..., 0])
Ptst = xgb.predict(Xtst[..., 0])
Ptrn.shape, Ptst.shape

print(f1_score(np.argmax(Ytrn, axis=1), Ptrn, average="macro"))
print(f1_score(np.argmax(Ytst, axis=1), Ptst, average="macro"))

1.0
0.8394904458598726
1.0
0.5664886326539337
